# N period Binomial model
## Backward Dynamic Programming

- enter a stock price today

- enter u, d, r, K - model parameters

- trace out the tree - iterate such that at each up and down from the previous step - we have an up and down in the next step. So at each time step - we have double the amount of nodes.

- after tracing out the path of the stock based on U and D - we calclate the option payoffs at the last time step. 

- we then run a backward program - iterating in the reverse direction from the last time step at each successive iteration then, we take the expectation with respect to RNP

In [3]:
import numpy as np

In [187]:
# a set of print commands that let the user enter his/her MODEL PARAMETERS

print("===")
print("Enter your model parameters: ")
print("===")
print()
S0 = int(input("enter today's stock price: "))
print()
K = int(input("enter the call option strike price: "))
print()
t = int(input("enter the number of time period (n period binomial model): "))
ts = t+1
print()
u = float(input("enter the stock UP ratio: "))
print()
d = float(input("enter the stock DOWN ratio: "))
print()
r = int(input("enter the risk free rate: "))
print()
print()
print()
print("YOUR MODEL PARAMETERS ARE: (S0: {}, K: {}, timestep: {}, up: {}, down: {}, rf: {})".format(
      S0, K, t, u, d, r))

q = (np.exp(r) - d)/(u-d)
print()
print("the RISK NEUTRAL probability for UP is: ", q)

===
Enter your model parameters: 
===



enter today's stock price:  100


enter the call option strike price:  91


enter the number of time period (n period binomial model):  10


enter the stock UP ratio:  2


enter the stock DOWN ratio:  0.5


enter the risk free rate:  0





YOUR MODEL PARAMETERS ARE: (S0: 100, K: 91, timestep: 10, up: 2.0, down: 0.5, rf: 0)

the RISK NEUTRAL probability for UP is:  0.3333333333333333


In [1]:
# this function defines the call option payoff and returns it

def call_option(ST, K):
    if ST > K:
        payoff = ST - K
    elif ST <= K:
        payoff = 0
    return payoff

In [4]:
# simulate the tree and number of nodes in each time step

def draw_tree(ts, S0):
    d1 = {}
    d2 = {}
    for t in range(ts):
        if t == 0:
            d1[t] = 1
        elif t > 0:
            d1[t] = d1[t-1]*2
        d2[t] = []

    d2[0].append(S0)
    
    return d1, d2

In [5]:
D1, D2 = draw_tree(3, 100)
D1

{0: 1, 1: 2, 2: 4}

In [6]:
# simulate the stock values of u and d movements at each time step - tracing out the path

def simulate_values(ts, D_1, D_2, u, d, K):
    for t in range(1, ts):
        for i in range(D_1[t-1]):
            D_2[t].append(D_2[t-1][i]*u)
            D_2[t].append(D_2[t-1][i]*d)
            
    D_2[len(D_2.keys())-1] = [call_option(x, K) for x in D_2[len(D_2.keys())-1]]
    return D_2

In [7]:
DD2 = simulate_values(3, D1, D2, 2, 0.5, 91)

In [8]:
DD2

{0: [100], 1: [200, 50.0], 2: [309, 9.0, 9.0, 0]}

In [9]:
# calculate the option payoff at each step in reverse

def backward_dynamic(dd, q):
    h = {}
    for t in sorted(list(dd.keys()), reverse=True):
        if t == 0:
            break

        arr = []
        for i in range(len(dd[t])):
            if i % 2 == 0:
                exp = np.exp(0)*(q*dd[t][i]) + ((1-q)*dd[t][i+1])
                arr.append(round(exp, 2))
            elif i % 2 != 0:
                pass

        h[t-1] = arr
        dd[t-1] = arr
        
    return h

In [10]:
OP = backward_dynamic(DD2, 0.33)

In [11]:
OP

{1: [108.0, 2.97], 0: [37.63]}

In [12]:
print("the option price TODAY is: ", OP[0])

the option price TODAY is:  [37.63]


In [13]:
OP[0]

[37.63]